# Generación de Datos Sintéticos - Proyecto de Grado
## Framework de RL para Optimización de Datos Sintéticos y Modelos ML

**Título:** Sintetizando datos tabulares: Un Framework de Aprendizaje por Refuerzo para el Benchmark de Datos Sintéticos y su Impacto en problemas de Clasificación

**Autor:** Carlos Andres Cortez Ballen

### Objetivos de este Notebook:
- Implementar y comparar generadores de datos sintéticos (GANs vs métodos tradicionales)
- Evaluar calidad de datos sintéticos usando métricas definidas en el proyecto
- Generar datos sintéticos para el segmento D (baja transaccionalidad)
- Preparar datos para entrenamiento de modelos ML
- Establecer baseline de calidad para datos sintéticos

### Metodología:
1. **Generadores GAN**: CTGAN, TVAE
2. **Generadores Tradicionales**: Gaussian Copula, SMOTE, ADASYN
3. **Métricas de Calidad**: KS Complement, Chi-Squared, KL Divergence, Cosine Similarity, Jensen-Shannon Entropy
4. **Validación**: 15-20% de datos para validación de sintetizadores

### Contenido:
1. Configuración y carga de datos
2. Implementación de generadores sintéticos
3. Entrenamiento de generadores
4. Generación de datos sintéticos
5. Evaluación de calidad
6. Comparación de generadores
7. Preparación para modelado


In [ ]:
# Importación de librerías y configuración
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
import logging
from tqdm import tqdm

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cargar configuración
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⚙️ Configuración cargada: {len(config)} secciones")


In [ ]:
# Importar módulos del proyecto
from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.data_splitter import DataSplitter
from src.synthetic_generators.generator_factory import GeneratorFactory
from src.synthetic_generators.quality_evaluator import SyntheticQualityEvaluator

# Inicializar componentes
data_loader = DataLoader(config)
data_preprocessor = DataPreprocessor(config)
data_splitter = DataSplitter(config)
generator_factory = GeneratorFactory(config)
quality_evaluator = SyntheticQualityEvaluator(config)

print("✅ Módulos del proyecto importados correctamente")
print("📁 DataLoader inicializado")
print("🔧 DataPreprocessor inicializado")
print("✂️ DataSplitter inicializado")
print("🏭 GeneratorFactory inicializado")
print("📊 QualityEvaluator inicializado")


In [ ]:
# Cargar datos del segmento D (usando German Credit como proxy)
print("🔄 Cargando datos del segmento D...")

try:
    # Cargar German Credit Data como proxy del segmento D
    german_features, german_targets = data_loader.load_uci_dataset(144, "german_credit")
    segment_d_data = pd.concat([german_features, german_targets], axis=1)
    
    print(f"✅ Datos del segmento D cargados: {segment_d_data.shape}")
    print(f"   Features: {german_features.shape[1]}")
    print(f"   Target distribution: {german_targets.iloc[:, 0].value_counts().to_dict()}")
    
    # Mostrar información básica
    print(f"\n📊 Información del dataset:")
    print(f"   Shape: {segment_d_data.shape}")
    print(f"   Memory usage: {segment_d_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Missing values: {segment_d_data.isnull().sum().sum()}")
    print(f"   Duplicates: {segment_d_data.duplicated().sum()}")
    
    # Mostrar primeras filas
    print(f"\n📖 Primeras 5 filas:")
    display(segment_d_data.head())
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    segment_d_data = None
